In [345]:
import pandas as pd
import torch
from torch import nn, optim
import numpy as np

In [346]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [347]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [348]:
y_train_df = train_data['Survived']
print(y_train_df.head())

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [349]:
def normalize(xi, range_min, range_max):
    return (xi - range_min) / (range_max - range_min)

In [350]:
x_train_df = train_data.PassengerId

# PClass to one hot encoding
x1_train_df = pd.get_dummies(train_data.Pclass)
x1_train_df = x1_train_df.set_axis([f'IsPClass{i}' for i in range(1, 4)], axis=1)
x_train_df = pd.concat([x_train_df, x1_train_df], axis=1)

# Gender to one hot encoding
x1_train_df = pd.get_dummies(train_data.Sex)
x1_train_df = x1_train_df.set_axis(['Female', 'Male'], axis=1)
x_train_df = pd.concat([x_train_df, x1_train_df], axis=1)

# Amount of siblings to continuous variable
x1_train_df = train_data[['SibSp']].copy()
x1_train_df = x1_train_df.apply(lambda x: normalize(x, min(x1_train_df.SibSp), max(x1_train_df.SibSp)), axis=0)
x_train_df = pd.concat([x_train_df, x1_train_df], axis=1)

x_train_df.head()

,PassengerId,IsPClass1,IsPClass2,IsPClass3,Female,Male,SibSp
0,1,0,0,1,0,1,0.125
1,2,1,0,0,1,0,0.125
2,3,0,0,1,1,0,0.000
3,4,1,0,0,1,0,0.125
4,5,0,0,1,0,1,0.000


In [351]:
# Dropping ID column
x_train_df = x_train_df.drop(columns=['PassengerId'], axis=0)
x_train_df.head()

,IsPClass1,IsPClass2,IsPClass3,Female,Male,SibSp
0,0,0,1,0,1,0.125
1,1,0,0,1,0,0.125
2,0,0,1,1,0,0.000
3,1,0,0,1,0,0.125
4,0,0,1,0,1,0.000


In [352]:
def create_nn(input_shape, hidden_shapes):
    assert len(hidden_shapes) > 0
    # Input Layer
    network_architecture = [nn.Linear(input_shape, hidden_shapes[0]), nn.ReLU()]

    # Hidden Layer
    hidden = [[nn.Linear(hidden_shapes[i], hidden_shapes[i + 1]), nn.ReLU()] for i in range(len(hidden_shapes) - 1)]
    for layer in hidden:
        for item in layer:
            network_architecture.append(item)

    # Output Layer
    network_architecture += [nn.Linear(hidden_shapes[-1], 1), nn.Sigmoid()]
    return nn.Sequential(*network_architecture)

In [353]:
input_size = x_train_df.shape[1]
hidden_layers = [32, 42, 62]
epochs = 100

print(f'Input size: {input_size}')
model = create_nn(input_size, hidden_layers)
print(model)

Input size: 6
Sequential(
  (0): Linear(in_features=6, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=42, bias=True)
  (3): ReLU()
  (4): Linear(in_features=42, out_features=62, bias=True)
  (5): ReLU()
  (6): Linear(in_features=62, out_features=1, bias=True)
  (7): Sigmoid()
)


In [354]:
x_train = []
y_train = []
for train_row in x_train_df.values:
    x_train.append(torch.tensor(train_row.astype(np.float32)))
for train_row in y_train_df.values:
    y_train.append(torch.tensor([train_row.astype(np.float32)]))

print(f'Input tensor features: {x_train_df.columns}')

Input tensor features: Index(['IsPClass1', 'IsPClass2', 'IsPClass3', 'Female', 'Male', 'SibSp'], dtype='object')


In [355]:
optimizer = optim.Adam(model.parameters(), lr=0.0003)
loss_func = nn.MSELoss()

for e in range(epochs + 1):

    running_loss = 0
    correct = 0
    wrong = 0

    for x, y in zip(x_train, y_train):
        optimizer.zero_grad()
        output = model(x)

        if output[0] > 0.5 and y == 1 or output[0] < 0.5 and y == 0:
            correct += 1
        else:
            wrong += 1

        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if e % 10 == 0:
        avg_loss = running_loss / len(x_train)
        print(f'Episode: {e:04d}: {correct:04d} / {wrong:04d} - Loss: {avg_loss}')

Episode: 0000: 0659 / 0232 - Loss: 0.18598473113026015
Episode: 0010: 0712 / 0179 - Loss: 0.1430409213710434
Episode: 0020: 0714 / 0177 - Loss: 0.14268113491394557
Episode: 0030: 0714 / 0177 - Loss: 0.1423799322487412
Episode: 0040: 0714 / 0177 - Loss: 0.14210490046414484
Episode: 0050: 0714 / 0177 - Loss: 0.14184565992420173
Episode: 0060: 0714 / 0177 - Loss: 0.14166076282714388
Episode: 0070: 0715 / 0176 - Loss: 0.1413846986623929
Episode: 0080: 0715 / 0176 - Loss: 0.1412059630307303
Episode: 0090: 0715 / 0176 - Loss: 0.14101102325935166
Episode: 0100: 0714 / 0177 - Loss: 0.14087375250564496
